# initialize

In [ ]:
import polars as pl
import pulp
import matplotlib.pyplot as plt
import seaborn as sns

# read data

In [ ]:
df_mst_category = pl.read_csv("../data/mst_category.csv")
df_mst_gender = pl.read_csv("../data/mst_gender.csv")
df_mst_member = pl.read_csv("../data/mst_member.csv")
df_mst_parent = pl.read_csv("../data/mst_parent.csv")
df_mst_grade_category = pl.read_csv("../data/mst_grade_category.csv")
df_trn_parent = pl.read_csv("../data/trn_touban.csv")


# 前処理

In [ ]:
# STAFFを除く
df_mst_parent = df_mst_parent.with_columns( df_mst_parent["staff"].fill_null(0) )
df_mst_parent = df_mst_parent.filter( pl.col("staff") == 0 )
df_mst_parent

# 中間テーブル作成

In [ ]:
df_parent_attr = df_mst_parent.clone()
# df_parent_attr.merge() 
# こどもを紐づけて、カテゴリフラグを付与
df_parent_attr = df_parent_attr.join(df_mst_member, on="parent_id", how="inner")

dict_gender = {0: "m", 1: "f"}
dict_cat = {0: "top", 1: "mid", 2: "und"}

for gender_k in dict_gender:
  for cat_k in dict_cat:  
    df_parent_attr = df_parent_attr.with_columns(
      pl.when( (pl.col("gender") == gender_k) & (pl.col("category") == cat_k)) 
      .then(1)
      .otherwise(0)
      .alias(f"{dict_gender[gender_k]}_{dict_cat[cat_k]}") 
    )
  for grade in range(1, 7):
    df_parent_attr = df_parent_attr.with_columns(
      pl.when( (pl.col("gender") == gender_k) & (pl.col("grade") == grade)) 
      .then(1)
      .otherwise(0)
      .alias(f"{dict_gender[gender_k]}_{grade}") 
    )
df_parent_attr = df_parent_attr.select([
  "parent_id", "parent_name", "staff", "beginner",
  "m_top","m_mid","m_und","m_1","m_2","m_3","m_4","m_5","m_6","f_top","f_mid","f_und","f_1","f_2","f_3","f_4","f_5","f_6"])
df_parent_attr = df_parent_attr.group_by(["parent_id", "parent_name", "staff", "beginner"], maintain_order=True).sum()
df_parent_attr.write_csv("../data/_intermediate_parent_attr.csv")
df_parent_attr

# input読み込み

In [ ]:
# 入力
# 12月の予定
df_input = pl.read_csv("../data/input.csv")
df_input

# optimize

## 仕様

* input
  * 計算したい月下記を与える
    * 日
    * 当番の人数
    * 性別ごとのカテゴリまたは学年
* output
  * 各練習に当番の人の名前
* 制約
  * 当番の担当の数は年間で平準化する (合計担当数の最小化)
  * 初心者はペアにしない
  * 役員とコーチは当番対象としてカウントしない

In [ ]:
prob = pulp.LpProblem("opt", sense=pulp.const.LpMinimize)

parents = df_mst_parent["parent_id"].to_list()
days = df_input["id"].to_list()
cats = df_mst_grade_category["cat_name"].to_list()

# 決定変数
parent_days = [(day, parent) for day in days for parent in parents ]
x = pulp.LpVariable.dicts("x", parent_days, cat=pulp.LpBinary)

# 最大値と最小値を表す補助変数を定義
x_count_by_parents = pulp.LpVariable.dict("x_count_by_parent", parents, cat=pulp.LpInteger, lowBound=0)
# x_count_by_parents = []
# for parent in parents:
#   x_count_by_parents.append(
#     pulp.LpVariable(f"x_count_by_parent{parent}", lowBound=0)
#   )
max_count = pulp.LpVariable("max_count", lowBound=0, cat=pulp.const.LpInteger)
max_count.setInitialValue(0)
min_count = pulp.LpVariable("min_count", lowBound=0)

# 目的関数
# 最大値最小化問題
prob.setObjective(
  max_count
)

# 制約条件
# 0. 補助変数 (parentごとの今月担当回数)
for parent in parents:
  prob.addConstraint(
    x_count_by_parents[parent] == pulp.lpSum( x[day, parent] for day in days )
  )

# 0. 補助変数 (最大値)
for parent in parents:
  prob.addConstraint(
    x_count_by_parents[parent] <= max_count
  )

# # 0. 最大値最小化でうまくいかなかったので、上から抑えた
# for parent in parents:
#   prob.addConstraint(
#     x_count_by_parents[parent] <= 1
#   )

# 1. 各日の当番の人数は2名 (あとで人数を指定できるようにする)
for day in days:
  prob.addConstraint(
    pulp.lpSum(
      [ x[day, parent] for parent in parents ]
    ) == 2
  )

# 準備
dict_parent_cat_vs_flag = {}
for parent in parents:
  dict_parent_cat_vs_flag[parent] = {}
  for cat in cats:
    flag_for_parent_in_cat = df_parent_attr.filter(pl.col("parent_id") == parent).select(cat).sum().to_series().to_list()[0]
    dict_parent_cat_vs_flag[parent][cat] = flag_for_parent_in_cat

dict_day_cat_vs_flag = {}
for day in days:
    dict_day_cat_vs_flag[day] = {}
    for cat in cats:
      # 今日のカテゴリのフラグ
      flag_for_day_in_cat = df_input.filter( pl.col("id") == day ).select(cat).sum().to_series().to_list()[0]
      dict_day_cat_vs_flag[day][cat] = flag_for_day_in_cat
# print(dict_day_cat_vs_flag)

# 2. 各dayの該当カテゴリにのみアサイン
for day in days:
  for parent in parents:
    prob.addConstraint(
      x[day, parent] <= ( 
        dict_parent_cat_vs_flag[parent][cat] * dict_day_cat_vs_flag[day][cat]
        for cat in cats
      )
    )

# 3. 初心者は一人にしない


# 4. 同じ日に当番にしない


# 5. Staffは対象外
# 前処理で対応済

# ----------------------------------------
# solve
result = prob.solve()
print(pulp.LpStatus[result])

# ----------------------------------------
# 結果出力
dict_parent = dict(zip(df_mst_parent['parent_id'].to_list(), df_mst_parent['parent_name'].to_list()))
for day in days:
  touban = []
  for parent in parents:
    if x[day, parent].value() == 1:
      # print(f"{day} {parent}")  
      touban.append(dict_parent[parent])
  print(f"{day}: {touban}")

print(max_count.value())
print(min_count.value())

maxvavava = 0
for parent in parents:
  print(dict_parent[parent], x_count_by_parents[parent].value())
  if maxvavava < x_count_by_parents[parent].value():
    maxvavava = x_count_by_parents[parent].value()
print(f"max: {maxvavava}")
